In [3]:
from torch.autograd import functional
import torch 
import numpy as np 
import math 
import os 


In [37]:
def u(input):
    x,y,z=input
    return (x**2 * y**2 * x**2 ,x**2 +y**2 * z**2 +y , x*z*y**2+y*z*x**2)

In [14]:
a=torch.tensor([1.0,2.0,3.0],requires_grad=True)
b=u(*a)
b

(tensor(4., grad_fn=<MulBackward0>),
 tensor(39., grad_fn=<AddBackward0>),
 tensor(18., grad_fn=<AddBackward0>))

In [23]:
def ux(input):
    x,y,z=input
    return x**2 * y**2 * z**2 

In [72]:
c=torch.trace(functional.hessian(ux,torch.tensor([3.0,2.0,2.0],requires_grad=True),create_graph=True))
c

tensor(176., grad_fn=<TraceBackward0>)

In [28]:
torch.diag(c).sum()

tensor(176.)

In [29]:
torch.trace(c)

tensor(176.)

In [69]:
#la divergencia es la traza del jacobiano de modo que 
input_tensor=torch.tensor([1.0,2.0,3.0],requires_grad=True)
div=torch.trace(torch.stack(functional.jacobian(u,input_tensor,create_graph=True),axis=0))
div

tensor(59., grad_fn=<TraceBackward0>)

In [70]:
torch.autograd.grad(div, input_tensor)

(tensor([56., 39., 24.]),)

## Ecuaciones del equilibrio  
$$\nabla^2\bold{u} + \frac{1}{1-2\nu}\nabla(\nabla \cdot \bold{u}) = 0$$  

$$\nabla^2\bold{u} = (\nabla^2 u_x , \nabla^2 u_y,\nabla^2 u_z)$$

$$\nabla(\nabla \cdot \bold{u})=\nabla(\frac{\partial}{\partial x}u_x+\frac{\partial}{\partial y}u_y + \frac{\partial}{\partial z}u_z) = \left( \frac{\partial}{\partial x}(\frac{\partial}{\partial x}u_x+\frac{\partial}{\partial y}u_y + \frac{\partial}{\partial z}u_z) , \frac{\partial}{\partial y}(\frac{\partial}{\partial x}u_x+\frac{\partial}{\partial y}u_y + \frac{\partial}{\partial z}u_z), \frac{\partial}{\partial z}(\frac{\partial}{\partial x}u_x+\frac{\partial}{\partial y}u_y + \frac{\partial}{\partial z}u_z) \right)$$

Vamos a usar el hessiano y le jacobiano para hacer todo esto mejor con pytorch autograd.   

Entonces, sea $\bold{u}(x,y,z)$ la función que va ha imitar nuestra red neuronal. Es una función vectorial $\bold{u}(x,y,z)=(u_x,u_y,u_z)$.  
Sea $J\bold{u}$ el jacobiano de esta función con respecto a las entradas $(x,y,z)$. Y sea el hessiano de cada uno de los componentes $Hess \bold{u}_i$ con $i=x,y,z$   
El jacobiano por un lado, representa las derivadas de todas las salidas con respecto a todas las entradas.  
Y el Hessiano representa las derivadas segundas de un componente de la salida, con respecto a todas las combinaciones de las entradads.    


La forma de relacionar esto con lo anterior es como sigue:     
$$\nabla^2\bold{u} = (\nabla^2 u_x , \nabla^2 u_y,\nabla^2 u_z)= ( \text{tr}(\it{Hess \ u_x}),\text{tr}(\it{Hess \ u_y}),\text{tr}(\it{Hess \ u_z}))$$   

$$\nabla(\nabla \cdot \bold{u})=\nabla(\frac{\partial}{\partial x}u_x+\frac{\partial}{\partial y}u_y + \frac{\partial}{\partial z}u_z) = \nabla ( \text{tr}(J \ \bold{u}) ) $$

En resumen, las ecuaciones del equilibrio quedan (son 3 ):  

$$( \text{tr}(\it{Hess \ u_x}),\text{tr}(\it{Hess \ u_y}),\text{tr}(\it{Hess \ u_z})) + \nabla ( \text{tr}(J \ \bold{u}) ) = 0$$